In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timezone
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
import calendar
import yaml
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from pycaret.classification import *

In [2]:
pd.get_option("display.max_columns", None)
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

In [5]:
Load_10K_Records=True
currency_pairs = ["EUR_USD"]
timeframe = "H4"
price_char = "M"
price_com = "mid"

In [6]:
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}

In [7]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

In [8]:
for pair in currency_pairs:
    pricing_params = (
        ('instruments', pair),
    )
    response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/pricing'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()

In [9]:
response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/openPositions'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()

In [10]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)

In [11]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [12]:
if Load_10K_Records:
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
            
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    response=second_response
else:
    response=first_response['candles']

In [13]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [14]:
data = data.drop_duplicates()
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

## Simple Moving Average (SMA)

In [15]:
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(5)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(5)

## Moving Average Range

In [16]:
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

## Price Range

In [17]:
data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

In [18]:
data = data.drop_duplicates()
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [19]:
data.tail(3)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_10,SMA_20,F_SMA_10,F_SMA_20,col_1,col_2,col_3
9636,2021-12-17,02:00:00,2,351,4,Friday,2021-12-17T02:00:00.000000000Z,4358,1.13210,1.13490,1.13204,1.13484,1.13119,1.12950,0.00365,0.00534,-0.00274,0.00280,0.00280
9637,2021-12-17,06:00:00,6,351,4,Friday,2021-12-17T06:00:00.000000000Z,12994,1.13482,1.13486,1.13172,1.13344,1.13193,1.12976,0.00151,0.00368,0.00138,0.00004,0.00172
9638,2021-12-17,10:00:00,10,351,4,Friday,2021-12-17T10:00:00.000000000Z,8470,1.13342,1.13342,1.13116,1.13144,1.13212,1.12994,-0.00068,0.00150,0.00198,0.00000,0.00028


## Candlestick Number

In [20]:
candle_no = len(data) - 2
#candle_no = 9659

In [21]:
def viz(data):
    fig = go.Figure(data=[go.Candlestick(x=data['UTC_Time'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False)
    
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
    
    fig.show()

In [22]:
candle = data.iloc[candle_no-3:candle_no+2]

In [23]:
candle[['Date','f_time','Volume']]

,Date,f_time,Volume
9634,2021-12-16,18,5901
9635,2021-12-16,22,3329
9636,2021-12-17,2,4358
9637,2021-12-17,6,12994
9638,2021-12-17,10,8470


## Average True Range (ATR)

In [24]:
high_low = data['High'] - data['Low']
high_cp = np.abs(data['High'] - data['Close'].shift())
low_cp = np.abs(data['Low'] - data['Close'].shift())
df = pd.concat([high_low, high_cp, low_cp], axis=1)
true_range = np.max(df, axis=1)
data['ATR_14'] = true_range.rolling(14).mean()

## Stop Loss / TakeProfit

In [25]:
ATR = data.iloc[candle_no]['ATR_14']
CLOSED_PRICE = data.iloc[candle_no]['Close']
BUY_SL = (CLOSED_PRICE - ATR).round(5)
SELL_SL = (CLOSED_PRICE + ATR).round(5)
BUY_TP = (CLOSED_PRICE + ATR).round(5)
SELL_TP = (CLOSED_PRICE - ATR).round(5)

## Feature Selection

In [26]:
data = data[['col_1','col_2','col_3','F_SMA_10','F_SMA_20']]

In [27]:
data.head(1)

,col_1,col_2,col_3,F_SMA_10,F_SMA_20
0,-0.00566,0.00681,0.0057,0.00505,0.00547


## Model

In [28]:
def find_k_similar_candles(candle_id, dataset, k=5):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'brute') 
    model_knn.fit(dataset)
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i==0:
            display (pd.DataFrame(data.iloc[candle_id]).transpose())
        else:
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
                            dataset.iloc[indices.flatten()[i]]['col_1'],
                            dataset.iloc[indices.flatten()[i]]['col_2'],
                            dataset.iloc[indices.flatten()[i]]['col_3'],
                            dataset.iloc[indices.flatten()[i]]['F_SMA_10'],
                            dataset.iloc[indices.flatten()[i]]['F_SMA_20'],
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
                      'col_1',
                      'col_2',
                      'col_3',
                      'F_SMA_10',
                      'F_SMA_20',
                     ]
    
    return indices, distances

In [29]:
indices, distances = find_k_similar_candles (candle_no,data)

,col_1,col_2,col_3,F_SMA_10,F_SMA_20
9637,0.00138,0.00004,0.00172,0.00151,0.00368


## Final Action

In [30]:
indices = indices[0:1][0]

In [31]:
indices

array([9637, 6715, 8131, 8350, 6808])

In [32]:
recs = []

for indice in indices[1:5]:
    
    data = pd.read_csv(filename) 
    data = data.iloc[indice:indice+7]
    
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X)
    
    coeficient = (linear_regressor.coef_)
    
    if coeficient > 0:
        recs.append((r2_score(Y, y_pred).round(2)*100))
    else:
        recs.append((r2_score(Y, y_pred).round(2)*100) * -1)

In [33]:
data_unseen = pd.DataFrame ({'Rec1_Score': [recs[0]], 'Rec2_Score': [recs[1]], 'Rec3_Score':[recs[2]], 'Rec4_Score':[recs[3]]})

In [34]:
data_unseen

,Rec1_Score,Rec2_Score,Rec3_Score,Rec4_Score
0,-68.0,8.0,62.0,33.0


In [35]:
SAVED_FINAL_MODEL = load_model('EURUSD/14-12-2021_12-59_AM_knn_EURUSD')
new_prediction = predict_model(SAVED_FINAL_MODEL, data=data_unseen)
new_prediction.head()

Transformation Pipeline and Model Successfully Loaded


,Rec1_Score,Rec2_Score,Rec3_Score,Rec4_Score,Label,Score
0,-68.0,8.0,62.0,33.0,1,0.5473


In [36]:
KNN_Pre = new_prediction['Label']
KNN_Pre[0]

1

In [37]:
SAVED_FINAL_MODEL = load_model('EURUSD/14-12-2021_09-08_AM_gbc_EURUSD')
new_prediction = predict_model(SAVED_FINAL_MODEL, data=data_unseen)
new_prediction.head()

Transformation Pipeline and Model Successfully Loaded


,Rec1_Score,Rec2_Score,Rec3_Score,Rec4_Score,Label,Score
0,-68.0,8.0,62.0,33.0,1,0.7616


In [38]:
GBC_Pre = new_prediction['Label']
GBC_Pre[0]

1

In [43]:
pair

'EUR_USD'

In [39]:
BUY_SL

1.13028

In [40]:
BUY_TP

1.1366

In [41]:
SELL_SL

1.1366

In [42]:
SELL_TP

1.13028